## Оконные функции

**Задача 1.** Примените оконные функции к таблице products и с помощью ранжирующих функций упорядочьте все товары по цене — от самых дорогих к самым дешёвым. Добавьте в таблицу следующие колонки:

Колонку product_number с порядковым номером товара (функция ROW_NUMBER).
Колонку product_rank с рангом товара с пропусками рангов (функция RANK).
Колонку product_dense_rank с рангом товара без пропусков рангов (функция DENSE_RANK).
Не забывайте указывать в окне сортировку записей — без неё ранжирующие функции могут давать некорректный результат, если таблица заранее не отсортирована. Деление на партиции внутри окна сейчас не требуется. Сортировать записи в результирующей таблице тоже не нужно.

In [ ]:
SELECT product_id,
       name,
       price,
       row_number() OVER (ORDER BY price desc) as product_number,
       rank() OVER (ORDER BY price desc) as product_rank,
       dense_rank() OVER (ORDER BY price desc) as product_dense_rank
FROM   products

**Задача 2.** Примените оконную функцию к таблице products и с помощью агрегирующей функции в отдельной колонке для каждой записи проставьте цену самого дорогого товара. Колонку с этим значением назовите max_price.

Затем для каждого товара посчитайте долю его цены в стоимости самого дорогого товара — просто поделите одну колонку на другую. Полученные доли округлите до двух знаков после запятой. Колонку с долями назовите share_of_max.

Выведите всю информацию о товарах, включая значения в новых колонках. Результат отсортируйте сначала по убыванию цены товара, затем по возрастанию id товара.

In [ ]:
SELECT product_id,
       name,
       price,
       max (price) OVER () as max_price,
       round(price/max (price) OVER (), 2) as share_of_max
FROM   products
ORDER BY 3 desc, 1

**Задача 3.** Примените две оконные функции к таблице products. Одну с агрегирующей функцией MAX, а другую с агрегирующей функцией MIN — для вычисления максимальной и минимальной цены. Для двух окон задайте инструкцию ORDER BY по убыванию цены. Поместите результат вычислений в две колонки max_price и min_price.

Выведите всю информацию о товарах, включая значения в новых колонках. Результат отсортируйте сначала по убыванию цены товара, затем по возрастанию id товара.

In [ ]:
SELECT product_id,
       name,
       price,
       max (price) OVER (ORDER BY price desc) as max_price,
       min (price) OVER (ORDER BY price desc) as min_price
FROM   products
ORDER BY 3 desc, 1

**Задача 4.** Сначала на основе таблицы orders сформируйте запрос, который вернет таблицу с общим числом заказов по дням. При подсчёте числа заказов не учитывайте отменённые заказы (их можно определить по таблице user_actions). Колонку с днями назовите date, а колонку с числом заказов — orders_count.

Затем поместите полученную таблицу в подзапрос и примените к ней оконную функцию в паре с агрегирующей функцией SUM для расчёта накопительной суммы числа заказов. Не забудьте для окна задать инструкцию ORDER BY по дате.

Колонку с накопительной суммой назовите orders_count_cumulative. В результате такой операции значение накопительной суммы для последнего дня должно получиться равным общему числу заказов за весь период.

Сортировку результирующей таблицы делать не нужно.

In [ ]:
SELECT date,
       orders_count,
       sum (orders_count) OVER (ORDER BY date)::int as orders_count_cumulative
FROM   (SELECT creation_time::date as date,
               count(order_id) as orders_count
        FROM   orders
        WHERE  order_id not in (SELECT order_id
                                FROM   user_actions
                                WHERE  action = 'cancel_order')
        GROUP BY date) t

**Задача 5.** Для каждого пользователя в таблице user_actions посчитайте порядковый номер каждого заказа.

Для этого примените оконную функцию ROW_NUMBER, используйте id пользователей для деления на патриции, а время заказа для сортировки внутри патриции. Отменённые заказы не учитывайте.

Новую колонку с порядковым номером заказа назовите order_number. Результат отсортируйте сначала по возрастанию id пользователя, затем по возрастанию порядкового номера заказа.

Добавьте в запрос оператор LIMIT и выведите только первые 1000 строк результирующей таблицы.

In [ ]:
SELECT user_id,
       order_id,
       time,
       row_number () OVER (PARTITION BY user_id
                           ORDER BY time) as order_number
FROM   user_actions
WHERE  order_id not in (SELECT order_id
                        FROM   user_actions
                        WHERE  action = 'cancel_order')
ORDER BY 1, 4 limit 1000

**Задача 6.** Дополните запрос из предыдущего задания и с помощью оконной функции для каждого заказа каждого пользователя рассчитайте, сколько времени прошло с момента предыдущего заказа. 

Для этого сначала в отдельном столбце с помощью LAG сделайте смещение по столбцу time на одно значение назад. Столбец со смещёнными значениями назовите time_lag. Затем отнимите от каждого значения в колонке time новое значение со смещением (либо можете использовать уже знакомую функцию AGE). Колонку с полученным интервалом назовите time_diff. Менять формат отображения значений не нужно, они должны иметь примерно следующий вид:

3 days, 12:18:22

По-прежнему не учитывайте отменённые заказы. Также оставьте в запросе порядковый номер каждого заказа, рассчитанный на прошлом шаге. Результат отсортируйте сначала по возрастанию id пользователя, затем по возрастанию порядкового номера заказа.

Добавьте в запрос оператор LIMIT и выведите только первые 1000 строк результирующей таблицы.

**Пояснение:**

Не забывайте про деление на партиции и сортировку внутри окна.

Также обратите внимание, что в результате смещения для первых заказов каждого пользователя в колонке time_lag получились пропущенные значения. Для таких записей функция не нашла предыдущих значений и вернула NULL. То же самое произошло в записях пользователей с одним заказом — внутри партиции с одной записью просто некуда сдвигаться.

Образовавшиеся пропущенные значения убирать из результата не нужно.

In [ ]:
SELECT user_id,
       order_id,
       time,
       row_number () OVER (PARTITION BY user_id
                           ORDER BY time) as order_number,
       lag (time, 1) OVER (PARTITION BY user_id
                           ORDER BY time) as time_lag,
       time-lag (time, 1) OVER (PARTITION BY user_id
                                ORDER BY time) as time_diff
FROM   user_actions
WHERE  order_id not in (SELECT order_id
                        FROM   user_actions
                        WHERE  action = 'cancel_order')
ORDER BY 1, 4 limit 1000

**Задача 7.** На основе запроса из предыдущего задания для каждого пользователя рассчитайте, сколько в среднем времени проходит между его заказами. Посчитайте этот показатель только для тех пользователей, которые за всё время оформили более одного неотмененного заказа.

Среднее время между заказами выразите в часах, округлив значения до целого числа. Колонку со средним значением времени назовите hours_between_orders. Результат отсортируйте по возрастанию id пользователя.

Добавьте в запрос оператор LIMIT и включите в результат только первые 1000 записей.

In [ ]:
with t as (SELECT user_id,
                  order_id,
                  time,
                  lag (time, 1) OVER (PARTITION BY user_id
                                      ORDER BY time) as time_lag,
                  time-lag (time, 1) OVER (PARTITION BY user_id
                                           ORDER BY time) as time_diff,
                  count (order_id) OVER (PARTITION BY user_id) as cnt
           FROM   user_actions
           WHERE  order_id not in (SELECT order_id
                                   FROM   user_actions
                                   WHERE  action = 'cancel_order'))
SELECT user_id,
       avg(extract(epoch
FROM   time_diff)/3600)::int as hours_between_orders
FROM   t
WHERE  cnt > 1
GROUP BY user_id
ORDER BY 1 limit 1000

**Задача 8.** Сначала на основе таблицы orders сформируйте новую таблицу с общим числом заказов по дням. При подсчёте числа заказов не учитывайте отменённые заказы (их можно определить по таблице user_actions). Колонку с числом заказов назовите orders_count.

Затем поместите полученную таблицу в подзапрос и примените к ней оконную функцию в паре с агрегирующей функцией AVG для расчёта скользящего среднего числа заказов. Скользящее среднее для каждой записи считайте по трём предыдущим дням. Подумайте, как правильно задать границы рамки, чтобы получить корректные расчёты.

Полученные значения скользящего среднего округлите до двух знаков после запятой. Колонку с рассчитанным показателем назовите moving_avg. Сортировку результирующей таблицы делать не нужно.

**Пояснение:**

При решении задачи можете пробовать разные границы рамки и проверять себя вручную. Важно для каждой даты учесть в расчётах именно 3 предыдущих дня.

Заполнять пропущенные значения в этой задаче не нужно. Подумайте, почему они могли появиться.

По-прежнему не учитывайте отменённые заказы.

In [ ]:
with t as (SELECT creation_time::date as date,
                  count(order_id) as orders_count
           FROM   orders
           WHERE  order_id not in (SELECT order_id
                                   FROM   user_actions
                                   WHERE  action = 'cancel_order')
           GROUP BY date)
SELECT date,
       orders_count,
       round(avg(orders_count) OVER(ORDER BY date rows between 3 preceding and 1 preceding),
             2) as moving_avg
FROM   t

**Задача 9.** Отметьте в отдельной таблице тех курьеров, которые доставили в сентябре 2022 года заказов больше, чем в среднем все курьеры.

Сначала для каждого курьера в таблице courier_actions рассчитайте общее количество доставленных в сентябре заказов. Затем в отдельном столбце с помощью оконной функции укажите, сколько в среднем заказов доставили в этом месяце все курьеры. После этого сравните число заказов, доставленных каждым курьером, со средним значением в новом столбце. Если курьер доставил больше заказов, чем в среднем все курьеры, то в отдельном столбце с помощью CASE укажите число 1, в противном случае укажите 0.

Колонку с результатом сравнения назовите is_above_avg, колонку с числом доставленных заказов каждым курьером — delivered_orders, а колонку со средним значением — avg_delivered_orders. При расчёте среднего значения округлите его до двух знаков после запятой. Результат отсортируйте по возрастанию id курьера.

In [ ]:
SELECT courier_id,
       delivered_orders,
       round(avg(delivered_orders) OVER(), 2) as avg_delivered_orders,
       case when delivered_orders > round(avg(delivered_orders) OVER(), 2) then 1
            else 0 end as is_above_avg
FROM   (SELECT courier_id,
               count(order_id) as delivered_orders
        FROM   courier_actions
        WHERE  (time >= '2022-09-01'
           and time < '2022-10-01')
           and action = 'deliver_order'
        GROUP BY courier_id) t
ORDER BY 1